# ra3 gene tree generation, Aug. 4, 2022

# Part 1: Making the tree

### Step 1: make a list of ra3 (T6PP) seed genes

Created a fasta file with 12 ra3-like genes from maize, sorghum, setaria, rice, and pineapple. The gene IDs were from a previous ra3 tree made by Jarrett. I looked up the peptide sequences in phytozome and built a fasta file in a text editor.

### Step 2: Start a new interactive session from the terminal.

`bsub -Is -q interactive -W 8:00 -n 1 -R "span[hosts=1]" bash`
    
Run mafft on the seed gene fasta file to get a multiple alignment file.

`module load MAFFT/7.313`

`mafft-linsi seed_genes.txt > seed_genes.aln`

In [1]:
#the resulting file
!cat seed_genes.aln

>Zm00001eb327910_P001
MTKHAAYSSEDVVAAVA--APAPAGRHFTSFQALKGAPL------DCKK--HAAVDLS--
---AS------GAAVVGGGPWFESMKASSPRRAAD---------------AEHGDW----
----MEKHPSALAQFEPLLAAAKGKQIVMFLDYDGTLSPIVEDPDRAVMSEEMREAVRRV
AEHFPTAIVSGRCRDKVLNFVKLTELYYAGSHGMDIQGPAACRQPNHVQQVVHTQ-AEA-
AAVHYQAASEFLPVIEE------------------VFRTLTAKMESIAGARVEHNKYCLS
VHFRCVREEEWNAVNEEVRSVLREYPNLKLTHGRKVRRHRTCWRFVRPSSGTRARPSSSC
SSLLAMLGATTSS-RFTSEMIALTRT----------------------LSRCSATWGRAS
ESWCPSFLRR-----RRHPTR----------------
>Zm00001d006913_P001 Zm00001d006913_P001 pep chromosome:AGPv4:2:219293809:219297189:1 gene:Zm00001d006913 transcript:Zm00001d006913_T001 gene_biotype:protein_coding transcript_biotype:protein_coding description:Trehalose 6-phosphate phosphatase [Source:Un
MTKHTAFAGADGGTTAA--AA-------VTLCAPPRARG------ARRV-----------
----------------AAGSLPEL----VRRHA------------------DLDDW----
----MEKHPSALAGFESVLAAAEGKQVVMFLDYDGTLSPIVKDPDSAVMSEEMRDAVRGV
AEHFPTAIVSGRCRDKVFNFVKLAELYYAGSHGMDIKGPTAQ--------SKHTK-AKA-
EAVLCQPASAFLP

### Step 3: Use HMMer to make a profile hidden markov model
` module load hmmer/3.1b2`

`hmmbuild -o summary.txt model.hmm seed_genes.aln`

### Step 4: Use the hmm to pull the top hits out of grass protein database.

First concatenate all the species protein files

`cat *.proteins.fasta > Poaceae.combined.fasta`

`hmmsearch -o hmmout.txt --noali --tblout search.output.txt model.hmm genomes/Poaceae/Poaceae.combined.fasta`


### Step 5: Extract all hmm search matches with higher score than outgroup

In [101]:
#remove first 3 header lines
!sed '1,3d' search.output.txt > search.output.mod.txt

In [75]:
#print up to outgroup, Aco004091
#!sed '/Aco004091/q' search.output.mod.txt > search.output.mod2.txt

In [102]:
#check that it worked
!cat search.output.mod.txt

Et_s16546-0.4-1.mrna1     -          seed_genes           -           3.2e-232  776.6   0.1  6.8e-115  390.6   0.0   2.0   1   1   1   2   2   2   2 pep supercontig:ASM97063v1:scaffold16546:5812:9050:1 gene:Et_s16546-0.4-1.path1 transcript:Et_s16546-0.4-1.mrna1 gene_biotype:protein_coding transcript_biotype:protein_coding
Et_s7243-0.22-1.mrna1     -          seed_genes           -           1.7e-229  767.6   3.2  6.8e-146  492.6   0.1   2.0   2   0   0   2   2   2   2 pep supercontig:ASM97063v1:scaffold7243:63610:67486:-1 gene:Et_s7243-0.22-1.path1 transcript:Et_s7243-0.22-1.mrna1 gene_biotype:protein_coding transcript_biotype:protein_coding
Zm00001eb327910_P002      -          seed_genes           -             1e-203  682.8   0.1  1.2e-203  682.5   0.1   1.0   1   0   0   1   1   1   1 -
SbRio.02G400100.1.p       -          seed_genes           -           2.4e-203  681.6   0.0  2.7e-203  681.4   0.0   1.0   1   0   0   1   1   1   1 pacid=38980196 transcript=SbRio.02G400100.1 locus=

In [ ]:
#read through the file until reaching the outgroup, extact each first column from line with awk
#look up the full protein sequences in Poaceae.combined.fasta and build a new fasta file for alignment

In [118]:
%%bash
#this isnt working through jupyter, skip to next block to run this as a script "buildNewGeneList.sh"

outgroup="Aco004091"
geneCount=0
#module load samtools/1.9
#samtools faidx Poaceae.combined.fasta #index combined fasta

cat search.output.mod.txt | while read line
do
    currentGene=$(echo $line | awk '{print $1}')
    echo $currentGene
    #samtools faidx Poaceae.combined.fasta $currentGene -o newGenes.fa #use samtools to extract match
    (( geneCount++ ))
    if [ "$currentGene" == $outgroup ]; then echo $geneCount; break; fi
done

#69 genes in first pass

Et_s16546-0.4-1.mrna1
Et_s7243-0.22-1.mrna1
Zm00001eb327910_P002
SbRio.02G400100.1.p
Sobic.002G381400
Sevir.2G407500
Seita.2G396100
SbRio.02G400400.1.p
Sobic.002G381600
SbRio.02G400300.1.p
Sobic.002G381500
Misin03G340000.1.p
Misin03G340100.1.p
OEL36545.1
Pahal.2G452900.1.p
Zm00001eb327900_P001
PhHAL.2G439600.1.p
PhHAL.2G439700.1.p
Misin04G363900.1.p
Seita.2G396200
Pahal.2G453000.1.p
ELECO.r07.7BG0609280.1
Pavir.2NG596120.1.p
Pavir.2KG544600.1.p
Pavir.2NG596100.2.p
ELECO.r07.7BG0609270.1
Aco009575
ELECO.r07.7AG0577950.1
Pavir.2KG544700.3.p
TVU37795
TraesCS2B02G187100.1
Traes_2BS_A5BF49CE9.1
Thint.05G0044700.1.p
Thint.01G0520500.1.p
Sevir.2G407600
KAF8679237.1
TraesCS2A02G161100.1
Zm00008a029404_P01
Thint.04G0155800.1.p
Thint.06G0409500.1.p
Thint.01G0520700.1.p
KAF8679238.1
Ata_020197684.1
Platifolius.Pl03g17530.1
Platifolius.Pl03g17530.1
Platifolius.Pl07g04800.1
Platifolius.Pl07g04800.1
strangu_016836-RA
strangu_016836-RA
Ata_020197683.1
TraesCS2D02G168200.1
TVU37789
HVMOREX.r2.2HG01016

in terminal do
`bash buildNewGeneList.sh`

In [108]:
!cat newGeneList.fa

>Et_s16546-0.4-1.mrna1
RKHPSALGKFEQIAGASKGKKIVMFLDYDGTLSPIVADPDAAYMSDAMRAAVRDVAKHFP
TAIVSGRCLDKVCNFVSLSELYYAGSHGMDIKGPSSNPESVLCQPASEFLPVIDEVYKAL
VEKTKSTPGAKVENNKFCLSVHFRCVDEKRWNALAEQVKAVTKDYPMLKLTQGRKVLEIR
PSIMWDKGKALEFLLESLGTCVATNHLAFVSSVAKLFPTALVTGRCLEKVYNFVGLSELY
YAGSHGMDIKGPSSNPESVLCQPASHFLPVIDEVYKALVEKTKSTPGAKVENNKFCLSVH
FRCVDEKRWNALAEQVKAVIKDYPMLKLTQGRKVLEIRPSIMWDKGKALEFLLESLGFAN
SSDVLPVYIGDDRTDEDAFKVLRKRGQGFGILVSKCPKETNASYSLRDPNEVMEFLVRLV
EWKRRSSSPMIRPRV
>Et_s7243-0.22-1.mrna1
SPPPRALPPPPGVAASAMRGNKYLQAQMEQHLAKAAPGRKINGLLESMRASSPTHAKAAA
ALAEERAAWMAKHPSALAKFEQVVAASKGKQIVVFLDYDGTLSPIVDDPDAAYMSDTMRR
AVRSVAKHFPTAIVSGRCRDKVFEFVKLAELYYAGSHGMDIKGPAKASRHTKAKAKRVLF
QPASEFLPMIEQNRVARSSLAQTKANLLPSDRLEQELDPCPDDEAERGGAPRALPSAGVA
ARGRKYLRAQMDHQHIAAPGRKINGLVESAAAAAALAAATGAVDDEERQAWMAKHPSALA
RFEQVVAASKGKQIVVFLDYDGTLSPIVDDPDAAYMSDTMRRAVRSVAKHFPTAIVSGRC
RDKVFEFVKLAELYYAGSHGMDIKGPAKASRHTKAKAKRVLFQPASEFLPMIQQVHESLI
EKTKCIPGAKVENNKFCVSVHFRCVDEKSWGTLADLVKSVLKDYPKLKLTQGRMVFEVRP
TIKWDKGKALEFLLESLGFADCA

### Step 6: rebuild the hmm model, run the search and collect new matches up to the outgroup (repeat steps 1-5), and then realign one last time using MAFFT

`bsub -Is -q interactive -W 8:00 -n 1 -R "span[hosts=1]" bash`

Run mafft on the seed gene fasta file to get a multiple alignment file.

`module load MAFFT/7.313`

`mafft-linsi newGeneList.fa > newGeneList.aln`

In [109]:
!cat newGeneList.aln

>Et_s16546-0.4-1.mrna1
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
-----------------------------------RKHPSALGKFEQIAGASKGKKIVMF
LDYDGTLSPIVADPDAAYMSDAMRAAVRDVAKHFPTAIVSGRCLD---------------
------------------------------------------------------------
----------------------------KVCNFVSLSELYYAGSHGMDIKGPSS--NPES
VLCQPASEFLPVIDEVYKALVEKTKSTPGAKVENNKFCLSVHFRCVDEKRWNALAEQVKA
VTKDYPMLKLTQGRKVLEIRPSIMWDKGKALEFLLESLGTCVATNHLAFVSSVAKLFPTA
LVTGRCLEKVYNFVGLSELYYAGSHGMDIKGPSSN--------------------PESVL
CQPASHFLPVIDE------------------VYKALVEKTKSTPGAKVENNKFCLSVHFR
CVDEKRWNALAEQVKAVIKDYPM--LKLTQGRKVLEIRPSIMWDKGKALEFLLESLGFAN
SS-DVLPVYIGDDRTDEDAFKVLRKRGQGFGILVSKCPKETNASYSLRDPNEVMEFLVRL
V

`module load hmmer/3.1b2`

`hmmbuild -o summary2.txt model2.hmm newGeneList.aln`

`hmmsearch -o hmmout2.txt --noali --tblout search.output2.txt model2.hmm Poaceae.combined.fasta`

In [111]:
!cat search.output2.txt

#                                                                    --- full sequence ---- --- best 1 domain ---- --- domain number estimation ----
# target name             accession  query name           accession    E-value  score  bias   E-value  score  bias   exp reg clu  ov env dom rep inc description of target
#     ------------------- ---------- -------------------- ---------- --------- ------ ----- --------- ------ -----   --- --- --- --- --- --- --- --- ---------------------
Et_s16546-0.4-1.mrna1     -          newGeneList          -           3.9e-252  842.1   0.1  2.3e-127  431.7   0.0   2.0   1   1   1   2   2   2   2 pep supercontig:ASM97063v1:scaffold16546:5812:9050:1 gene:Et_s16546-0.4-1.path1 transcript:Et_s16546-0.4-1.mrna1 gene_biotype:protein_coding transcript_biotype:protein_coding
Et_s7243-0.22-1.mrna1     -          newGeneList          -           1.2e-251  840.5   3.1  6.2e-164  552.0   0.1   2.0   2   0   0   2   2   2   2 pep supercontig:ASM97063v1:scaffold7

In [119]:
#remove first 3 header lines
!sed '1,3d' search.output2.txt > search.output2.mod.txt

In [6]:
%%bash
#this isnt working through jupyter, skip to next block to run this as a script "buildNewGeneList2.sh"

outgroup="Aco004091"
geneCount=0
#module load samtools/1.9
#samtools faidx Poaceae.combined.fasta #index combined fasta

cat search.output2.mod.txt | while read line
do
    currentGene=$(echo $line | awk '{print $1}')
    echo $currentGene
    echo $currentGene >> ra3TreeGeneList2.txt
    #samtools faidx Poaceae.combined.fasta $currentGene -o newGenes.fa #use samtools to extract match
    (( geneCount++ ))
    if [ "$currentGene" == $outgroup ]; then echo $geneCount; break; fi
done

#200 genes in second pass

Et_s16546-0.4-1.mrna1
Et_s7243-0.22-1.mrna1
TraesCS2B02G187100.1
Traes_2BS_A5BF49CE9.1
Thint.05G0044700.1.p
TraesCS2A02G161100.1
Ata_020197684.1
Thint.01G0520500.1.p
TraesCS2B02G187200.1
Thint.04G0155800.1.p
Thint.06G0409500.1.p
Ata_020197683.1
Thint.01G0520700.1.p
TraesCS2D02G168200.1
TraesCS2A02G161000.1
TraesCS2A02G161200.1
HVMOREX.r2.2HG0101600.1
TraesCS2D02G168300.1
Ata_020188443.1
TraesCS2D02G168100.1
TraesCS2B02G187000.1
Traes_2BS_29C388771.1
ELECO.r07.7BG0609280.1
Sevir.2G407500
PhHAL.2G439700.1.p
Pahal.2G453000.1.p
Seita.2G396100
OEL36545.1
Traes_2AS_7DFCB009C.1
TraesCS2D02G168300.2
PhHAL.2G439600.1.p
Pahal.2G452900.1.p
Zm00001eb327910_P002
SbRio.02G400100.1.p
Sobic.002G381400
Pavir.2NG596120.1.p
Pavir.2KG544600.1.p
KAF8679237.1
Platifolius.Pl07g04800.1
Platifolius.Pl07g04800.1
strangu_016836-RA
strangu_016836-RA
Misin03G340000.1.p
Platifolius.Pl03g17530.1
Platifolius.Pl03g17530.1
Pavir.2NG596100.2.p
SbRio.02G400400.1.p
Sobic.002G381600
Misin03G340100.1.p
TVU37795
Seita.2G3962

In [ ]:
!cat ra3TreeGeneList2.txt

in terminal do `bash buildNewGeneList2.sh` to get the new fasta file

Last step before tree generation- align using mafft:

`bsub -Is -q interactive -W 8:00 -n 1 -R "span[hosts=1]" bash`

`module load MAFFT/7.313`

`mafft-linsi newGeneList2.fa > newGeneList2.aln`



### Step 7: Run raxml-ng to generate a tree


`module load raxml-ng/0.9.0`

`bsub -q long -W 8:00 -n 4 -o "raxml_job.out" -e "raxml_job.err" -R "span[hosts=1]" -R "rusage[mem=2000]" singularity exec $RAXMLNGIMG raxml-ng-mpi --msa newGeneList2.fa --model JTT+G --threads 4`

In [1]:
#view tree
!cat newGeneList2.aln.raxml.bestTree

(SbRio.02G400400.1.p:0.000001,Sobic.002G381600:0.000001,(((((TVU37789:0.173295,ELECO.r07.7BG0609280.1:0.114264):0.076993,(((((Thint.01G0520700.1.p:0.025901,(Thint.01G0520600.1.p:0.175371,Thint.01G0520500.1.p:0.005887):0.010104):0.047023,(HVMOREX.r2.2HG0101600.1:0.110184,((((((((TraesCS2B02G187200.1:0.008904,((Ata_020197684.1:0.000001,TraesCS2D02G168300.1:0.000001):0.000001,TraesCS2D02G168300.2:0.004854):0.015967):0.033890,TraesCS2A02G161200.1:0.022699):0.042902,TRIUR3_27278-P1:0.147568):0.000001,TraesCS2A02G161100.1:0.017392):0.000001,(Thint.06G0409500.1.p:0.030772,(TraesCS2D02G168200.1:0.004307,Ata_020197683.1:0.000001):0.026892):0.013020):0.029070,(TraesCS2B02G187100.1:0.000001,Traes_2BS_A5BF49CE9.1:0.000001):0.016624):0.021091,(Thint.04G0155600.1.p:0.043482,(((Traes_2AS_7DFCB009C.1:0.000001,TraesCS2A02G161000.1:0.000001):0.021635,((Traes_2BS_29C388771.1:0.000001,TraesCS2B02G187000.1:0.000001):0.037080,(Thint.06G0409700.1.p:0.000001,(Ata_020188443.1:0.000001,TraesCS2D02G168100.1:0.00

In [ ]:
#install toytree to look at trees in notebook
!pip install toytree

In [4]:
import sys
#tell python to look in your local folder for the newly installed packages
sys.path.append('/home/ad32a/.local/lib/python3.9/site-packages')
#import the new packages to python
import toytree
import toyplot
import numpy as np

In [5]:
tre = toytree.tree("newGeneList2.aln.raxml.bestTree")
#draw tree
tre = tre.root("Aco004091") #root the tree
canvas, axes, mark = tre.draw(width=4000, height=5000, tip_labels_style={"font-size": "30px"});


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="4000.0px" height="5000.0px" viewBox="0 0 4000.0 5000.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tec9ad9deb85a4bffb8c2f6fc4bff76e9"> PhHAL.2G236900.1.p Pahal.2G245500.1.p Pavir.2NG308500.2.p Pavir.2KG329800.1.p Sevir.2G195900 Seita.2G188200 KAF8648043.1 KAF8688721.1 OEL37787.1 Misin04G171600.1.p Misin03G153200.1.p Sobic.002G184600 SbRio.02G195500.1.p ELECO.r07.6BG0486590.1 ELECO.r07.6AG0533550.1 Oropetium_20150105_05922A TVU09337 TVT99806 Ata_020190227.1 TraesCS5D02G200800.1 Traes_5DL_6AF673654.2 TraesCS5A02G190000.1 TRIUR3_29736-P1 Thint.14G0312300.1.p Traes_5BL_8F67396AE.2 TraesCS5B02G193100.3 TraesCS5B02G193100.1 Thint.13G0258100.1.p Thint.15G0314700.1.p HVMOREX.r2.5HG0394730.1 Bradi4g29030 Osat.015651449.1 LOC_Os09g20390 OMERI09G06230.1 LPERR09G05430.1 Ola014572.1 Platifolius.Pl09g08580.1 Pavir.6NG121400.1.p Pavir.6KG241500.1.p Pahal.6G190600.1.p PhHAL.6G181000.1.p Seita.6G141700 Sevir.6G146700 SbRio.07G132900.1.p Sobic.007G124200 Misin13G097200.1.p Misin07G319500.1.p Zm00001eb042260_P003 OEL17207.1 KAF8697396.1 ELECO.r07.8BG0658400.1 ELECO.r07.8AG0629610.1 TVU06116 Et_s7243-0.22-1.mrna1 KN540693.1_FGP001 LOC_Os08g31630 Osat.015650924.1 ORGLA08G0125300.1 LPERR08G11760.1 Bradi3g35590 Ola009149.1 Platifolius.Pl08g08820.1 strangu_006279-RA strangu_006279-RB strangu_032117-RA Aco014723 Sobic.004G232900 SbRio.04G248100.1.p Misin08G307100.1.p Zm00001eb255170_P001 Zm00008a023024_P01 Pahal.1G399900.1.p PhHAL.1G392100.1.p Pavir.1NG497100.1.p Pavir.1KG264552.1.p KAF8729896.1 Seita.1G323700 Sevir.1G330000 OEL22698.1 Et_s16546-0.4-1.mrna1 Et_s2019-0.44-1.mrna1 TVU28562 TVU28551 ELECO.r07.2AG0146090.1 Oropetium_20150105_04238A LOC_Os02g51680 Osat.015623803.1 ORGLA02G0283400.1 OMERI02G31420.1 OB02G39760.1 LPERR02G26870.1 TraesCS6D02G281100.1 Ata_020155875.1 TraesCS6B02G330900.2 TraesCS6A02G301800.1 Thint.16G0375000.1.p HVMOREX.r2.6HG0509380.1 Thint.18G0613400.1.p Thint.17G0507400.1.p Thint.16G0507600.1.p Thint.17G0500100.1.p Bradi3g58960 Ola031630.1 Platifolius.Pl02g28950.1 strangu_025916-RA TraesCS2D02G168300.1 Ata_020197684.1 TraesCS2D02G168300.2 TraesCS2B02G187200.1 TraesCS2A02G161200.1 TRIUR3_27278-P1 TraesCS2A02G161100.1 Ata_020197683.1 TraesCS2D02G168200.1 Thint.06G0409500.1.p Traes_2BS_A5BF49CE9.1 TraesCS2B02G187100.1 TraesCS2D02G168100.1 Ata_020188443.1 Thint.06G0409700.1.p TraesCS2B02G187000.1 Traes_2BS_29C388771.1 TraesCS2A02G161000.1 Traes_2AS_7DFCB009C.1 Thint.05G0044700.1.p Thint.04G0155600.1.p Thint.04G0155800.1.p Thint.05G0044800.1.p HVMOREX.r2.2HG0101600.1 Thint.01G0520500.1.p Thint.01G0520600.1.p Thint.01G0520700.1.p Bradi1g21420 LOC_Os07g43160 Osat.015645668.1 OMERI07G19170.1 LPERR07G19750.1 OB07G28260.1 Rgu014325 PhHAL.2G439700.1.p Pahal.2G453000.1.p Pavir.2KG544600.1.p Pavir.2NG596120.1.p Seita.2G396100 Sevir.2G407500 KAF8679237.1 KAF8668800.1 OEL36545.1 Sobic.002G381600 SbRio.02G400400.1.p Misin03G340000.1.p Zm00001eb327910_P002 ELECO.r07.7BG0609280.1 TVU37789 SbRio.02G400100.1.p Sobic.002G381400 SbRio.02G400300.1.p Sobic.002G381500 Zm00008a029404_P01 Zm00001eb327900_P001 Misin03G340100.1.p Misin04G363900.1.p Zm00001eb109730_P001 Pahal.2G452900.1.p PhHAL.2G439600.1.p Pavir.2NG596100.2.p Pavir.2KG544700.3.p Sevir.2G407600 Seita.2G396200 OEL36548.1 KAF8668799.1 KAF8679238.1 ELECO.r07.7AG0577950.1 ELECO.r07.7BG0609270.1 TVU37795 strangu_016836-RA Platifolius.Pl07g04800.1 PhHAL.9G457400.1.p Pahal.9G445600.1.p Pavir.9KG377800.1.p Sevir.9G386700 ELECO.r07.3BG0277730.1 ELECO.r07.3AG0250420.1 TVU46791 Bradi1g60950 Ola035599.1 strangu_025991-RA Platifolius.Pl03g17530.1 Aco009575 Aco004091

In [23]:
#save as pdf
import toyplot.pdf
toyplot.pdf.render(canvas, "Ra3_GeneTree_Best_ML_tree.pdf")

# Part 2: mapping CNS character states to tree

### Step 1: extract RA3 orthologs conserved at specified deep CNS regions

first identify the deeply conserved CNS region for RA3 in grasses. 
Deeply conserved CNS regions are identified using Conservatory.
In conservatory/alignments/ there is a bam file with all the conserved alignments. Use samtools to extract the aligned
reads from the different RA3 grass orthologs for the CNS region. Pipe to awk to just keep the ortholog name.

`module load samtools/1.9`

`samtools view Poaceae.bam chr7:176043000-176043020 | awk '{print $1}' | awk -F: '{print $3}' > ra3_176043000_orthos.txt`

In [2]:
!cat ra3_176043000_orthos.txt

SbRio.02G400300
ELECO.r07.7BG0609280
Pavir.2KG544600
KAF8668800
KAF8679237
KAF8775104
Pavir.2NG596120
OEL36545
PhHAL.2G439700
Thint.04G0155800
Misin03G340000
TVU37789
Seita.2G396100
Sevir.2G407500
TraesCS2A02G161100
TraesCS2B02G187100
Thint.04G0155600
Thint.05G0044700
Thint.06G0409500
TRIUR3_27278-P1
TraesCS2B02G187000


### Step 2: Use pandas to build a dataframe of CNS presence/absence of all RA3 tree genes

In [1]:
import pandas as pd

In [13]:
df = pd.read_csv('ra3TreeGeneList2.txt', header=None).drop_duplicates() #read in and remove duplicates
df.rename(columns={0:'ra3_tree_genes'}, inplace=True)
df = df.reset_index(drop=True)
df

,ra3_tree_genes
0,Et_s16546-0.4-1.mrna1
1,Et_s7243-0.22-1.mrna1
2,TraesCS2B02G187100.1
3,Traes_2BS_A5BF49CE9.1
4,Thint.05G0044700.1.p
...,...
185,HVMOREX.r2.6HG0509380.1
186,Thint.17G0507400.1.p
187,Thint.16G0507600.1.p
188,TVU09337


In [10]:
from numpy import loadtxt
cns2 = loadtxt("ra3_176043000_orthos.txt", comments="#", delimiter="/n", unpack=False, dtype="str")
cns2

array(['SbRio.02G400300', 'ELECO.r07.7BG0609280', 'Pavir.2KG544600',
       'KAF8668800', 'KAF8679237', 'KAF8775104', 'Pavir.2NG596120',
       'OEL36545', 'PhHAL.2G439700', 'Thint.04G0155800', 'Misin03G340000',
       'TVU37789', 'Seita.2G396100', 'Sevir.2G407500',
       'TraesCS2A02G161100', 'TraesCS2B02G187100', 'Thint.04G0155600',
       'Thint.05G0044700', 'Thint.06G0409500', 'TRIUR3_27278-P1',
       'TraesCS2B02G187000'], dtype='<U20')

In [4]:
cns1 = loadtxt("ra3_176042874_orthos.txt", comments="#", delimiter="/n", unpack=False, dtype="str")
cns1

array(['Misin03G340000', 'OEL36545', 'SbRio.02G400300', 'Seita.2G396100',
       'Sevir.2G407500', 'ELECO.r07.7BG0609280', 'Pavir.2KG544600',
       'Thint.04G0155600', 'Thint.04G0155800', 'TraesCS2A02G161100',
       'TraesCS2B02G187100', 'Thint.05G0044700', 'Thint.06G0409500',
       'Thint.06G0409700', 'TRIUR3_26378-P1', 'TRIUR3_27278-P1',
       'KAF8668800', 'KAF8679237', 'KAF8775104'], dtype='<U20')

In [5]:
cns3 = loadtxt("ra3_176045674_orthos.txt", comments="#", delimiter="/n", unpack=False, dtype="str")
cns3

array(['Seita.2G396100', 'TVU37789', 'Misin03G340000', 'PhHAL.2G439700',
       'Pavir.2NG596120', 'Misin03G340100', 'OEL36545', 'SbRio.02G400300',
       'SbRio.02G400100', 'Oropetium_20150105_10517A', 'KAF8668799',
       'KAF8679238', 'TVU37795', 'Platifolius.Pl03g17530', 'TVU46791',
       'Bradi1g60950', 'KAF8698624', 'KAF8668800', 'KAF8679237',
       'KAF8728855', 'KAF8775104', 'TVU46791', 'Et_s5558-0.0-1',
       'Et_s5558-0.0-1', 'Ola035599', 'Pavir.2KG544600', 'Rgu009072',
       'Sevir.2G407500', 'Misin04G363900', 'LPERR07G19750',
       'Misin02G324200', 'SbRio.01G377200', 'Seita.9G381600',
       'Sevir.9G386700', 'PhHAL.9G457400', 'Pavir.9KG377800',
       'Seita.2G396200', 'Sevir.2G407600', 'OEL36548',
       'ELECO.r07.7AG0577950', 'ELECO.r07.7BG0609270', 'PhHAL.2G439600',
       'Pavir.2KG544700', 'LPERR03G16920', 'OB03G28930', 'OB07G28260',
       'ORGLA03G0185400', 'Osat.015628649.1'], dtype='<U25')

In [6]:
cns4 = loadtxt("ra3_176072382_orthos.txt", comments="#", delimiter="/n", unpack=False, dtype="str")
cns4

array(['SbRio.02G400300', 'Misin03G340000', 'Seita.2G396100',
       'Sevir.2G407500', 'TraesCS2B02G187100', 'TraesCS2A02G161200',
       'Thint.04G0155600', 'PhHAL.2G439700', 'OEL36545', 'TVU37789',
       'TraesCS2B02G187200', 'Pavir.2NG596120', 'Pavir.2KG544700',
       'KAF8668800', 'KAF8775104', 'LPERR07G19750', 'PhHAL.2G439600',
       'KAF8679237', 'Ata_020188443', 'TraesCS2B02G187000'], dtype='<U18')

In [14]:
import re
df['CNS1-56074']=df['ra3_tree_genes'].str.contains('|'.join(map(re.escape, cns1)))
df['CNS2-56231']=df['ra3_tree_genes'].str.contains('|'.join(map(re.escape, cns2)))
df['CNS3-58875']=df['ra3_tree_genes'].str.contains('|'.join(map(re.escape, cns3)))
df['CNS4-85583']=df['ra3_tree_genes'].str.contains('|'.join(map(re.escape, cns4)))

In [15]:
df

,ra3_tree_genes,CNS1-56074,CNS2-56231,CNS3-58875,CNS4-85583
0,Et_s16546-0.4-1.mrna1,False,False,False,False
1,Et_s7243-0.22-1.mrna1,False,False,False,False
2,TraesCS2B02G187100.1,True,True,False,True
3,Traes_2BS_A5BF49CE9.1,False,False,False,False
4,Thint.05G0044700.1.p,True,True,False,False
...,...,...,...,...,...
185,HVMOREX.r2.6HG0509380.1,False,False,False,False
186,Thint.17G0507400.1.p,False,False,False,False
187,Thint.16G0507600.1.p,False,False,False,False
188,TVU09337,False,False,False,False


In [17]:
df.to_csv("ra3_conservedCNS.csv")

In [18]:
!cat ra3_conservedCNS.csv

,ra3_tree_genes,CNS1-56074,CNS2-56231,CNS3-58875,CNS4-85583
0,Et_s16546-0.4-1.mrna1,False,False,False,False
1,Et_s7243-0.22-1.mrna1,False,False,False,False
2,TraesCS2B02G187100.1,True,True,False,True
3,Traes_2BS_A5BF49CE9.1,False,False,False,False
4,Thint.05G0044700.1.p,True,True,False,False
5,TraesCS2A02G161100.1,True,True,False,False
6,Ata_020197684.1,False,False,False,False
7,Thint.01G0520500.1.p,False,False,False,False
8,TraesCS2B02G187200.1,False,False,False,True
9,Thint.04G0155800.1.p,True,True,False,False
10,Thint.06G0409500.1.p,True,True,False,False
11,Ata_020197683.1,False,False,False,False
12,Thint.01G0520700.1.p,False,False,False,False
13,TraesCS2D02G168200.1,False,False,False,False
14,TraesCS2A02G161000.1,False,False,False,False
15,TraesCS2A02G161200.1,False,False,False,True
16,HVMOREX.r2.2HG0101600.1,False,False,False,False
17,TraesCS2D02G168300.1,False,False,False,False
18,Ata_020188443.1,False,False,False,True
19,TraesCS2D02G168100.1,False,False,False,False
20,TraesCS2B0

### Step 3: Use R phytools to map character states to tree

In [ ]:
#ra3Treegen.R (in google drive)